## DM Project
## Muhammad Maemoon Farooq
## 21i1680
## DS N

Libraries

In [ ]:
!pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from prophet import Prophet
from tensorflow.keras.layers import LSTM, Dense
from sklearn.svm import SVR
import statsmodels.api as sm



## Loading the csv's

In [ ]:
AEP_df = pd.read_csv('AEP_hourly.csv')
CO2_df=pd.read_csv('Daily_atmospheric_CO2_concentration.csv')
sp500_df=pd.read_csv('sp500_index.csv')

In [ ]:
AEP_df.describe()

In [ ]:
CO2_df.describe()

In [ ]:
sp500_df.describe()

In [ ]:
counts=AEP_df.isnull().sum()
counts

In [ ]:
counts=sp500_df.isnull().sum()
counts

In [ ]:
counts=CO2_df.isnull().sum()
counts

##Scaling the dataset

In [ ]:
def scale_time_series(df, column_name, scaler_type='min_max'):
    values = df[column_name].values.reshape(-1, 1)

    if scaler_type == 'min_max':
        scaler = MinMaxScaler()
    elif scaler_type == 'standard':
        scaler = StandardScaler()
    else:
        raise ValueError("Invalid scaler_type. Use 'min_max' or 'standard'.")

    scaled_values = scaler.fit_transform(values)

    if scaler_type == 'min_max':
        new_column_name = f'{column_name}_min_max_scaled'
    elif scaler_type == 'standard':
        new_column_name = f'{column_name}_standard_scaled'

    df[new_column_name] = scaled_values

    return df


In [ ]:
AEP_df = scale_time_series(AEP_df,column_name='AEP_MW', scaler_type='min_max')
AEP_df = scale_time_series(AEP_df,column_name='AEP_MW', scaler_type='standard')
AEP_df

In [ ]:
sp500_df = scale_time_series(sp500_df,column_name='S&P500', scaler_type='min_max')
sp500_df = scale_time_series(sp500_df,column_name='S&P500', scaler_type='standard')
sp500_df

In [ ]:
CO2_df = scale_time_series(CO2_df,column_name='day', scaler_type='min_max')
CO2_df = scale_time_series(CO2_df,column_name='day', scaler_type='standard')
CO2_df

In [ ]:
def stationarize_time_series(df, column_name):
    df['diff'] = df[column_name].diff()

# Replacing 0 with a very small number before taking log to avoid log(0)
    df['diff'].replace(0, np.finfo(float).eps, inplace=True)

# Applying logarithmic transformation on the absolute difference
    df['log'] = np.log(np.abs(df['diff']))

# Handling initial NaN values by forward filling or other methods if suitable
    df['log'].fillna(method='bfill', inplace=True)

    return df

In [ ]:
CO2_df = stationarize_time_series(CO2_df, column_name='day')
AEP_df = stationarize_time_series(AEP_df, column_name='AEP_MW')
sp500_df = stationarize_time_series(sp500_df, column_name='S&P500')

In [ ]:
sp500_df

In [ ]:
CO2_df

In [ ]:
AEP_df

# ARIMA Configuration and Tuning

In [ ]:
def adf_test(series):
    result = adfuller(series, autolag='AIC')
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))



In [ ]:
print("SP500 ADF test")
adf_test(sp500_df['log'])

fig, ax = plt.subplots(2, 1, figsize=(12, 8))
acf_vals = acf(sp500_df['log'])
pacf_vals = pacf(sp500_df['log'])
ax[0].stem(acf_vals, use_line_collection=True)
ax[0].set_title('ACF Plot')
ax[1].stem(pacf_vals, use_line_collection=True)
ax[1].set_title('PACF Plot')
plt.show()

model = ARIMA(sp500_df['log'], order=(1, 1, 1))
model_fit = model.fit()

print(model_fit.summary())

forecast = model_fit.forecast(steps=10)
print(forecast)

In [ ]:
print("AEP ADF test")
adf_test(AEP_df['log'])

fig, ax = plt.subplots(2, 1, figsize=(12, 8))
acf_vals = acf(AEP_df['log'])
pacf_vals = pacf(AEP_df['log'])
ax[0].stem(acf_vals, use_line_collection=True)
ax[0].set_title('ACF Plot')
ax[1].stem(pacf_vals, use_line_collection=True)
ax[1].set_title('PACF Plot')
plt.show()

model = ARIMA(AEP_df['log'], order=(1, 1, 1))
model_fit = model.fit()

print(model_fit.summary())

forecast = model_fit.forecast(steps=10)
print(forecast)

In [ ]:
print("CO2 ADF test")
adf_test(CO2_df['log'])

fig, ax = plt.subplots(2, 1, figsize=(12, 8))
acf_vals = acf(CO2_df['log'])
pacf_vals = pacf(CO2_df['log'])
ax[0].stem(acf_vals, use_line_collection=True)
ax[0].set_title('ACF Plot')
ax[1].stem(pacf_vals, use_line_collection=True)
ax[1].set_title('PACF Plot')
plt.show()

model = ARIMA(CO2_df['log'], order=(1, 1, 1))
model_fit = model.fit()

print(model_fit.summary())

forecast = model_fit.forecast(steps=10)
print(forecast)

## ANN Design and Training

Min-Max Column used to train neural network


In [ ]:
def train_neural_network(X_train, X_test, y_train, y_test, epochs=10, batch_size=32):

    # Defining the architecture
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
    ])

    # Compiling the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Training the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

    # Evaluating the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)

    y_pred = model.predict(X_test)
    y_pred_classes = (y_pred > 0.5).astype("int32")

    precision = precision_score(y_test, y_pred_classes)
    recall = recall_score(y_test, y_pred_classes)
    f1 = f1_score(y_test, y_pred_classes)

    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

    return model

In [ ]:
print("sp500 ANN")
features = sp500_df[['S&P500_min_max_scaled']]
target = sp500_df['log']

X = features.values
y = target.values

# Binarizing the target for classification
y_binary = (y > y.mean()).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

model = train_neural_network(X_train, X_test, y_train, y_test)


In [ ]:
print("CO2 ANN")
features = CO2_df[['day_min_max_scaled']]
target = CO2_df['log']

X = features.values
y = target.values

# Binarize the target for binary classification
y_binary = (y > y.mean()).astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Train the neural network
model = train_neural_network(X_train, X_test, y_train, y_test)


In [ ]:
print("AEP ANN ")
features = AEP_df[['AEP_MW_min_max_scaled']]  # or 'S&P500_standard_scaled'
target = AEP_df['log']

# Convert DataFrame columns to numpy arrays
X = features.values
y = target.values

# Binarize the target for binary classification
y_binary = (y > y.mean()).astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Train the neural network
model = train_neural_network(X_train, X_test, y_train, y_test)


## SARIMA (Seasonal ARIMA)

In [ ]:
# Assuming AEP_df is already loaded and processed
print("AEP Sarima Model")
AEP_df['Datetime'] = pd.to_datetime(AEP_df['Datetime'])
AEP_df.set_index('Datetime', inplace=True)

# Fill NaN values and prepare data
AEP_df['AEP_MW'].fillna(method='ffill', inplace=True)
AEP_df['AEP_MW'].fillna(method='bfill', inplace=True)

# Check for stationarity
def test_stationarity(timeseries, window=12):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries.dropna(), autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

test_stationarity(AEP_df['AEP_MW'])

# Limit the parameter space for auto_arima
smodel = auto_arima(AEP_df['AEP_MW'], seasonal=True, m=2,
                    trace=True, error_action='ignore', suppress_warnings=True,
                    stepwise=True, max_order=6, max_p=2, max_q=2, max_d=1,
                    max_P=1, max_Q=1, max_D=1)

print(smodel.summary())

# Fit SARIMA model using optimized parameters
model = SARIMAX(AEP_df['AEP_MW'], order=smodel.order, seasonal_order=smodel.seasonal_order,
                enforce_stationarity=False, enforce_invertibility=False)
results = model.fit(disp=False)
print(results.summary())

# Forecasting and plotting results
forecast = results.get_forecast(steps=12)
mean_forecast = forecast.predicted_mean
conf_int = forecast.conf_int()

plt.figure(figsize=(10, 5))
plt.plot(AEP_df.index, AEP_df['AEP_MW'], label='Observed')
plt.plot(mean_forecast.index, mean_forecast, color='red', label='Forecast')
plt.fill_between(mean_forecast.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.legend()
plt.title('Forecast vs Actuals')
plt.show()


In [ ]:
print("CO2 Sarima Model")
CO2_df['date'] = pd.to_datetime(CO2_df[['year', 'month', 'day']])
CO2_df.set_index('date', inplace=True)

target_column = 'cycle'

CO2_df[target_column].fillna(method='ffill', inplace=True)
CO2_df[target_column].fillna(method='bfill', inplace=True)

CO2_df[target_column].plot()
plt.title(f'Time Series Data after Handling NaNs - {target_column}')
plt.show()

def test_stationarity(timeseries, window=12):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries.dropna(), autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

test_stationarity(CO2_df[target_column])

# Determining the SARIMA parameters
smodel = auto_arima(CO2_df[target_column], seasonal=True, m=2, trace=True, error_action='ignore', suppress_warnings=True)
print(smodel.summary())

# Fiting SARIMA model using the parameters
model = SARIMAX(CO2_df[target_column],
                order=smodel.order,
                seasonal_order=smodel.seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False)
results = model.fit(disp=False)
print(results.summary())

# Forecasting values
forecast = results.get_forecast(steps=12)
mean_forecast = forecast.predicted_mean
conf_int = forecast.conf_int()

plt.figure(figsize=(10, 5))
plt.plot(CO2_df.index, CO2_df[target_column], label='Observed')
plt.plot(mean_forecast.index, mean_forecast, color='red', label='Forecast')
plt.fill_between(mean_forecast.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.legend()
plt.title(f'Forecast vs Actuals - {target_column}')
plt.show()


In [ ]:
# Ensure the 'Date' column is in datetime format and set as index
if 'Date' in sp500_df.columns:
    sp500_df['Date'] = pd.to_datetime(sp500_df['Date'])
    sp500_df.set_index('Date', inplace=True)
else:
    raise KeyError("The 'Date' column is not found in the DataFrame.")

# Choose the target column for time series analysis
target_column = 'S&P500'

# Fill NaN values
sp500_df[target_column].fillna(method='ffill', inplace=True)  # Forward fill
sp500_df[target_column].fillna(method='bfill', inplace=True)  # Backward fill

# Visualize the data to confirm no NaNs
sp500_df[target_column].plot()
plt.title(f'Time Series Data after Handling NaNs - {target_column}')
plt.show()

# Function to test stationarity
def test_stationarity(timeseries, window=12):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries.dropna(), autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

# Check stationarity
test_stationarity(sp500_df[target_column])

# Determine the SARIMA parameters automatically
smodel = auto_arima(sp500_df[target_column], seasonal=True, m=12, trace=True, error_action='ignore', suppress_warnings=True)
print(smodel.summary())

# Fit SARIMA model using the best found parameters
model = SARIMAX(sp500_df[target_column],
                order=smodel.order,
                seasonal_order=smodel.seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False)
results = model.fit(disp=False)
print(results.summary())

# Forecasting future values
forecast = results.get_forecast(steps=12)
mean_forecast = forecast.predicted_mean
conf_int = forecast.conf_int()

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot(sp500_df.index, sp500_df[target_column], label='Observed')
plt.plot(mean_forecast.index, mean_forecast, color='red', label='Forecast')
plt.fill_between(mean_forecast.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.legend()
plt.title(f'Forecast vs Actuals - {target_column}')
plt.show()

## ETS

In [ ]:
def apply_ets_model(df, column, trend, seasonal, seasonal_periods):
    train_size = int(len(df) * 0.8)
    train, test = df.iloc[:train_size], df.iloc[train_size:]

    # Handle missing values in the data
    train = train.dropna()
    test = test.dropna()

    ets_model = ExponentialSmoothing(train[column], trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods).fit()
    forecast = ets_model.forecast(len(test))

    mae = mean_absolute_error(test[column], forecast)
    mse = mean_squared_error(test[column], forecast)
    r2 = r2_score(test[column], forecast)

    plt.figure(figsize=(10, 6))
    plt.plot(train.index, train[column], label='Train')
    plt.plot(test.index, test[column], label='Test')
    plt.plot(test.index, forecast, label='Forecast')
    plt.legend()
    plt.title(f'{column} - ETS Model')
    plt.show()

    return mae, mse, r2




In [ ]:
print("AEP ETS Model:")
mae, mse, r2 = apply_ets_model(AEP_df, 'diff', trend='add', seasonal='add', seasonal_periods=365)
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")

In [ ]:
print("CO2 ETS Model:")
mae, mse, r2 = apply_ets_model(CO2_df, 'diff', trend='add', seasonal='add', seasonal_periods=365)
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")



In [ ]:
print("sp500 ETS Model:")
mae, mse, r2 = apply_ets_model(sp500_df, 'diff', trend='add', seasonal='add', seasonal_periods=365)
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")


## Prophet

In [ ]:

def apply_prophet_model(df, column):
    prophet_df = df.reset_index().rename(columns={df.index.name: 'ds', column: 'y'})
    train_size = int(len(prophet_df) * 0.8)
    train, test = prophet_df.iloc[:train_size], prophet_df.iloc[train_size:]

    prophet_model = Prophet()
    prophet_model.fit(train)
    forecast = prophet_model.predict(test[['ds']])

    mae = mean_absolute_error(test['y'], forecast['yhat'])
    mse = mean_squared_error(test['y'], forecast['yhat'])
    r2 = r2_score(test['y'], forecast['yhat'])

    prophet_model.plot(forecast)
    plt.title(f'{column} - Prophet Model')
    plt.show()

    return mae, mse, r2



In [ ]:
print("AEP Prophet Model:")
mae, mse, r2 = apply_prophet_model(AEP_df, 'diff')
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")



In [ ]:
print("CO2 Prophet Model:")
mae, mse, r2 = apply_prophet_model(CO2_df, 'diff')
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")



In [ ]:
print("sp500 Prophet Model:")
mae, mse, r2 = apply_prophet_model(sp500_df, 'diff')
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")


## Support Vector Regression (SVR)

In [ ]:

def apply_svr_model(df, column):
    df = df.dropna()
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[[column]])
    df['Scaled'] = scaled_data

    train_size = int(len(df) * 0.8)
    train, test = df.iloc[:train_size], df.iloc[train_size:]

    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)
    svr_model.fit(train.index.values.reshape(-1, 1), train['Scaled'])
    forecast = scaler.inverse_transform(svr_model.predict(test.index.values.reshape(-1, 1)).reshape(-1, 1)).flatten()

    mae = mean_absolute_error(test[column], forecast)
    mse = mean_squared_error(test[column], forecast)
    r2 = r2_score(test[column], forecast)

    plt.figure(figsize=(10, 6))
    plt.plot(train.index, train[column], label='Train')
    plt.plot(test.index, test[column], label='Test')
    plt.plot(test.index, forecast, label='Forecast')
    plt.legend()
    plt.title(f'{column} - SVR Model')
    plt.show()

    return mae, mse, r2



In [ ]:

print("AEP SVR Model:")
mae, mse, r2 = apply_svr_model(AEP_df, 'diff')
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")



In [ ]:
print("CO2 SVR Model:")
mae, mse, r2 = apply_svr_model(CO2_df, 'diff')
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")



In [ ]:
print("sp500 SVR Model:")
mae, mse, r2 = apply_svr_model(sp500_df, 'diff')
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")


## Long Short-Term Memory (LSTM)

In [ ]:


def apply_lstm_model(df, column, n_steps=10):
    df = df.dropna()
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[[column]])

    X, y = [], []
    for i in range(n_steps, len(scaled_data)):
        X.append(scaled_data[i-n_steps:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)

    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=50, verbose=0)

    forecast = model.predict(X_test)
    forecast = scaler.inverse_transform(forecast)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    mae = mean_absolute_error(y_test, forecast)
    mse = mean_squared_error(y_test, forecast)
    r2 = r2_score(y_test, forecast)

    plt.figure(figsize=(10, 6))
    plt.plot(df.index[:train_size+n_steps], df[column][:train_size+n_steps], label='Train')
    plt.plot(df.index[train_size+n_steps:], df[column][train_size+n_steps:], label='Test')
    plt.plot(df.index[train_size+n_steps:], forecast, label='Forecast')
    plt.legend()
    plt.title(f'{column} - LSTM Model')
    plt.show()

    return mae, mse, r2


In [ ]:

# Apply LSTM Model to each dataset
print("AEP LSTM Model:")
mae, mse, r2 = apply_lstm_model(AEP_df, 'diff')
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")



In [ ]:
print("CO2 LSTM Model:")
mae, mse, r2 = apply_lstm_model(CO2_df, 'diff')
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")


In [ ]:

print("sp500 LSTM Model:")
mae, mse, r2 = apply_lstm_model(sp500_df, 'diff')
print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")

## Hybrid-Model

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def apply_arima_model(df, column, order):
    # Split the data into train and test sets
    train_size = int(len(df) * 0.8)
    train, test = df[column][:train_size], df[column][train_size:]

    # Fit ARIMA model
    model = ARIMA(train, order=order)
    model_fit = model.fit()

    # Forecast
    forecast = model_fit.forecast(steps=len(test))

    # Calculate residuals
    residuals = test - forecast

    # Calculate metrics
    mae = mean_absolute_error(test, forecast)
    mse = mean_squared_error(test, forecast)
    r2 = r2_score(test, forecast)

    return forecast, residuals, mae, mse, r2



In [ ]:
# Define function to train ANN on ARIMA residuals
def train_ann_on_residuals(residuals, n_steps=10):
    residuals = residuals.dropna().values.flatten()
    if len(residuals) < n_steps:
        raise ValueError("Not enough residuals to train the ANN model after removing NaN values.")

    scaler = MinMaxScaler()
    residuals = scaler.fit_transform(residuals.reshape(-1, 1)).flatten()

    X, y = [], []
    for i in range(n_steps, len(residuals)):
        X.append(residuals[i-n_steps:i])
        y.append(residuals[i])

    X, y = np.array(X), np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=500)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    return predictions, mae, mse, r2

In [ ]:
# Define function to combine forecasts
def combine_forecasts(arima_forecast, ann_forecast):
    min_length = min(len(arima_forecast), len(ann_forecast))
    arima_forecast = arima_forecast[:min_length]
    ann_forecast = ann_forecast[:min_length]

    combined_forecast = (arima_forecast + ann_forecast) / 2
    return combined_forecast


In [ ]:
# Apply ARIMA model to AEP_df
print("AEP ARIMA Model:")
aep_forecast, aep_residuals, mae, mse, r2 = apply_arima_model(AEP_df, 'AEP_MW', (5,1,0))
print(f"ARIMA MAE: {mae}, ARIMA MSE: {mse}, ARIMA R-squared: {r2}")


In [ ]:
# Train ANN on ARIMA residuals for AEP_df
print("AEP ANN on ARIMA Residuals:")
try:
    aep_ann_forecast, mae, mse, r2 = train_ann_on_residuals(aep_residuals)
    print(f"ANN MAE: {mae}, ANN MSE: {mse}, ANN R-squared: {r2}")

    # Combine forecasts
    aep_combined_forecast = combine_forecasts(aep_forecast, aep_ann_forecast)
    print("AEP Combined Forecast:")
    print(aep_combined_forecast)


In [ ]:
 # Plot results
    plt.figure(figsize=(10, 6))
    plt.plot(AEP_df.index[-len(aep_combined_forecast):], AEP_df['AEP_MW'].values[-len(aep_combined_forecast):], label='Actual')
    plt.plot(AEP_df.index[-len(aep_combined_forecast):], aep_combined_forecast, label='Combined Forecast', color='red')
    plt.legend()
    plt.title('AEP Combined Forecast vs Actual')
    plt.show()
except ValueError as e:
    print(e)